In [3]:
# osconfeed.py：下载osconfeed.json

from urllib.request import urlopen
import warnings
import os
import json

URL = 'http://www.oreilly.com/pub/sc/osconfeed'
JSON = 'data/osconfeed.json'

def load():
    if not os.path.exists(JSON):
        msg = 'downloading {} to {}'.format(URL, JSON)
        warnings.warn(msg) 
        with urlopen(URL) as remote, open(JSON, 'wb') as local:
            local.write(remote.read())
    with open(JSON) as fp:
        return json.load(fp) 

In [4]:
feed = load()
sorted(feed['Schedule'].keys())
for key, value in sorted(feed['Schedule'].items()):
    print('{:3} {}'.format(len(value), key))

  1 conferences
  1 events
  1 speakers
  1 venues


In [5]:
feed['Schedule']['speakers'][-1]['name']

'Robert Lefkowitz'

In [6]:
feed['Schedule']['speakers'][-1]['serial']

157509

In [7]:
feed['Schedule']['events'][40]['name']

IndexError: list index out of range

### 19.1.1 使用动态属性访问JSON类数据

In [9]:
from collections import abc
from typing import Any

# 只读接口，使用属性表示法访问Json类对象
class FrozenJSON:
    def __init__(self, mapping):
        # 使用mapping参数构建一个字典。这么做有两个目的：
        # (1)确保传入的是字典（或者是能转换成字典的对象）；(2)安全起见，创建一个副本。
        self.__data = dict(mapping)
    def __getattr__(self, name):
        # 仅当没有指定名称（name）的属性时才调用__getattr__方法。
        if hasattr(self.__data, name):
            # 如果name是实例属性__data的属性，返回那个属性。调用keys等方法就是通过这种方式处理的。
            return getattr(self.__data, name)
        else:
            # 否则，从self.__data中获取name键对应的元素，返回调用FrozenJSON.build()方法得到的结果
            return FrozenJSON.build(self.__data[name])
    @classmethod
    # 这是一个备选构造方法，@classmethod装饰器经常这么用。
    def build(cls, obj):
        # 如果obj是映射，那就构建一个FrozenJSON对象
        if isinstance(obj, abc.Mapping):
            return cls(obj)
        elif isinstance(obj, abc.MutableSequence):
            # 如果是MutableSequence对象，必然是列表，
            return [cls.build(item) for item in obj]
        else:
            # 如果既不是字典也不是列表，那么原封不动地返回元素。
            return obj
        

### 19.1.2 处理无效属性名

In [11]:
grad = FrozenJSON({'name': 'Jan Xo', 'class': 2024})
# 此时无法读取grad.class的值，因为在Python中class是保留字
grad.class

SyntaxError: invalid syntax (941653597.py, line 2)

In [12]:
# 但是可以这样做
getattr(grad, 'class')

2024

In [16]:
# explore1.py：在名称为Python关键字的属性后面加上_

def __init__(self, mapping):
    self.__data = {}
    for key, value in mapping.items():
        if keyword.iskeyword(key):
            key+= '_'
        self.__data[key] = value

# 这种情况也不行
x = FrozenJSON({'2asd': 'orasd'})
x.2asd

SyntaxError: invalid syntax (728741840.py, line 11)

### 19.1.3 使用__new__方法以灵活的方式创建对象

In [ ]:
from collections import abc
class FrozenJSON:
    """一个只读接口，使用属性表示法访问JSON类对象
    """
    def __new__(cls, arg):
        if isinstance(arg, abc.Mapping):
            return super().__new__(cls) 
        elif isinstance(arg, abc.MutableSequence):
            return [cls(item) for item in arg]
        else:
            return arg
    def __init__(self, mapping):
        self.__data = {}
        for key, value in mapping.items():
            if iskeyword(key):
                key+= '_'
            self.__data[key] = value
    def __getattr__(self, name):
        if hasattr(self.__data, name):
            return getattr(self.__data, name)
        else:
            return FrozenJSON(self.__data[name])

### 19.1.4 使用shelve模块调整OSCON数据源的结构

In [ ]:
import warnings

DB_NAME = 'data/schedule1_db'
CONFERENCE = 'conference.115'
class Record:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
def load_db(db):
    raw_data = osconfeed.load()
    warnings.warn('loading '+DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        for record in rec_list:
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = Record(**record)

### 19.1.5 使用特性获取链接的记录

In [ ]:
class MissingDatabaseError(RuntimeError):
    """需要数据库但没有指定数据库时抛出。"""
class DbRecord(Record):
    __db = None
    @staticmethod
    def set_db(db):
        DbRecord.__db = db
    @staticmethod
    def get_db():
        return DbRecord.__db
    @classmethod
    def fetch(cls, ident):
        db = cls.get_db()
        try:
            return db[ident]
        except TypeError:
            if db is None:
                msg = "database not set; call '{}.set_db(my_db)'"
                raise MissingDatabaseError(msg.format(cls.__name__))
            else:
                raise
    def __repr__(self):
        if hasattr(self, 'serial'):
            cls_name = self.__class__.__name__
            return '<{} serial={!r}>'.format(cls_name, self.serial)
        else:
            return super().__repr__()

In [ ]:
class Event(DbRecord):  
    @property
    def venue(self):
        key = 'venue.{}'.format(self.venue_serial)
        return self.__class__.fetch(key) 
    @property
    def speakers(self):
        if not hasattr(self, '_speaker_objs'): 
            spkr_serials = self.__dict__['speakers'] 
            fetch = self.__class__.fetch 
            self._speaker_objs = [fetch('speaker.{}'.format(key))
                                  for key in spkr_serials] 
        return self._speaker_objs 
    def __repr__(self):
        if hasattr(self, 'name'):
            cls_name = self.__class__.__name__
            return '<{} {!r}>'.format(cls_name, self.name)
        else:
            return super().__repr__()

In [ ]:
def load_db(db):
    raw_data = osconfeed.load()
    warnings.warn('loading '+DB_NAME)
    for collection, rec_list in raw_data['Schedule'].items():
        record_type = collection[:-1]
        cls_name = record_type.capitalize() 
        cls = globals().get(cls_name, DbRecord) 
        if inspect.isclass(cls) and issubclass(cls, DbRecord): 
            factory = cls 
        else:
            factory = DbRecord 
        for record in rec_list: 
            key = '{}.{}'.format(record_type, record['serial'])
            record['serial'] = key
            db[key] = factory(**record) 